In [1]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [2]:
from Main_vb import main_solver
import numpy as np
import time
import pickle
import pandas as pd
import os

# import numpy file from sampler
numpy_sampler = False
single_sample = True

path = '..\\01_SamplingFeatures'
if numpy_sampler:
        # name = 'data_240724_1752_case4\outfile.npy'
        name = 'data_20241104_1545_case8\outfile.npy'
        features = np.load(os.path.join(path, name))
else: 
        name = 'output\\data_20241111_0904_case10\\outfile.pkl'
        with open(os.path.join(path, name),'rb') as handle:
                in_dict = pickle.load(handle)


t0 = time.time()

# import mat_res.pkl from data that was used for training of algorithm
path_mat_res = '..\\02_Simulator'
# name = 'Simulator\\results\saved_runs\data_240805_1134_case4\mat_res.pkl'        # take 240805 here, as there the SN are included
# name = 'Simulator\\results\saved_runs\data_20241104_1854_case8\mat_res.pkl'
name = 'Simulator\\results\saved_runs\data_20241111_1101_case10\mat_res.pkl'
with open(os.path.join(path_mat_res, name),'rb') as handle:
        mat_res = pickle.load(handle)

# define location of trained model and input data to be used
path_train = '..\\04_Training'

data_path = {
        # "_": os.path.join(path_train,'new_data\\_simple_logs\\v_80'),
        "_I": os.path.join(path_train,'new_data\\_simple_logs\\v_198'),
        "_II": os.path.join(path_train,'new_data\\_simple_logs\\v_198'),
        "_III": os.path.join(path_train,'new_data\\_simple_logs\\v_198'),
}
model_path = {
        # "D": os.path.join(path_train,'logs\\train_log\\version_83_main\\checkpoints\\best_model.ckpt'),
        # "m": os.path.join(path_train,'logs\\train_log\\version_128\\checkpoints\\best_model_m.ckpt'),
        # "b": os.path.join(path_train,'logs\\train_log\\version_128\\checkpoints\\best_model_b.ckpt'),
        # "s": os.path.join(path_train,'logs\\train_log\\version_128\\checkpoints\\best_model_s.ckpt'),
        #"sig": os.path.join(path_train, 'new_data\\_simple_logs\\v_80\\best_trained_model__5138.pt'),
        "sig_I": os.path.join(path_train, 'new_data\\_simple_logs\\v_198\\best_trained_model__9997.pt'),
        "sig_II": os.path.join(path_train, 'new_data\\_simple_logs\\v_198\\best_trained_model__9997.pt'),
        "sig_III": os.path.join(path_train, 'new_data\\_simple_logs\\v_198\\best_trained_model__9997.pt'),
}

path_collection = {
        "model": model_path,
        "data": data_path
}

-------------------------------------------------------
Start Executing FEM_Q Script
-------------------------------------------------------


Seed set to 42


In [3]:
# Inputs for iteration

# If the data comes from the sampler:
if numpy_sampler:
    mat_tot_dict = {
        'L': features[:,0],         # length
        'B': features[:,1],         # width
        'E_1': features[:,2],       # Young's modulus steel / glass / concrete
        'E_2': features[:,3],       # Young's modulus - / interlayer / reinforcing steel
        'ms': features[:,4],        # mesh_size
        'F': features[:,5],         # force_magnitude
        's': features[:,6],         # scenario 0...6
        't_1': features[:,7],       # thickness of the plate
        't_2': features[:,8],       # thickness of plate
        'nl': features[:,9],        # amount of layers
        'nu_1': features[:,10],     # Poisson's ratio
        'nu_2': features[:,11],     # Poisson's ratio
        'mat': features[:,12]       # Material type     (1 = lin.el., 3 = CMM, 10 = glass)
    }
elif numpy_sampler == False: 
    mat_tot_dict = in_dict

if single_sample:
    # # pure shear force (scenario 10)
    # mat_tot_dict = {
    #     'L': np.array([7500]),
    #     'B': np.array([7500]),
    #     'E_1': np.array([33600]),
    #     'E_2': np.array([0]),
    #     'ms': np.array([750]),
    #     'F': np.array([750000]),       #=10*L
    #     'F_N': np.array([0]),         # not required in this case
    #     's': np.array([8]),
    #     't_1': np.array([200]),
    #     't_2': np.array([0]),
    #     'nl': np.array([20]),
    #     'nu_1': np.array([0.2]),
    #     'nu_2': np.array([0]),
    #     'mat': np.array([1])
    # }

    # # moment + normal force (scenario 11)
    # mat_tot_dict = {
    #     'L': np.array([7500]),
    #     'B': np.array([7500]),
    #     'E_1': np.array([33600]),
    #     'E_2': np.array([0]),
    #     'ms': np.array([750]),
    #     'F': np.array([0.015]),           # Uniformly distributed load in z-direction
    #     'F_N': np.array([750000]),       # Normal force (n[N/m]*L); e.g. n=50kN/m --> F_N = 100*7500 = 750000
    #     's': np.array([11]),
    #     't_1': np.array([200]),
    #     't_2': np.array([0]),
    #     'nl': np.array([20]),
    #     'nu_1': np.array([0.2]),
    #     'nu_2': np.array([0]),
    #     'mat': np.array([1])
    # }

    # moment (scenario 10)
    # mat_tot_dict = {
    #     'L': np.array([7500]),
    #     'B': np.array([7500]),
    #     'E_1': np.array([33600]),
    #     'E_2': np.array([0]),
    #     'ms': np.array([750]),
    #     'F': np.array([0.015]),
    #     'F_N': np.array([0]),       # not required in this case.
    #     's': np.array([10]),
    #     't_1': np.array([200]),
    #     't_2': np.array([0]),
    #     'nl': np.array([20]),
    #     'nu_1': np.array([0.2]),
    #     'nu_2': np.array([0]),
    #     'mat': np.array([1])
    # }

    # bending 2D (for glass; scenario 20)
    mat_tot_dict = {
        'L': np.array([1500]),
        'B': np.array([1500]),
        'E_1': np.array([70000]),
        'E_2': np.array([300]),
        'ms': np.array([150]),
        'F': np.array([0.005]),
        'F_N': np.array([0]),       # not required in this case.
        's': np.array([20]),
        't_1': np.array([5]),
        't_2': np.array([0.4]),
        'nl': np.array([5]),
        'nu_1': np.array([0.23]),
        'nu_2': np.array([0.5]),
        'mat': np.array([10])
    }



mat_tot_raw = pd.DataFrame.from_dict(mat_tot_dict)


# Choose the simulation number(s) that shall be tested
# desired_SN = [21, 32, 45, 74, 76, 82]
desired_SN = [0]
mat_tot = mat_tot_raw.iloc[[np.where(mat_res['SN'] == i)[0][0] for i in desired_SN]]
mat_tot.reset_index(drop=True, inplace=True)
print('As a check, in mat_tot (from sampling) (t1_tot = ', round(mat_tot['t_1'][0], 1), 'mm) should be equal to t1 in mat_res (directly read) (t1_res = ', round(mat_res['t_1'][desired_SN[0]],1), 'mm).')
# print(mat_tot)

conv_plt = True
simple = True                          # Leave this on "True" for deployment.
NN_hybrid = {
    'predict_D': False,                 # If true, solves the system with NN_hybrid solver (prediction of D); if False: "normal" solver
    'predict_sig': True               # If true, solves the system with NN_hybrid solver (prediction of sig); if False: "normal" solver
    }
# Note: predict_D should not be used in lin.el. case, as there is only one initialisation and this happens with lin.el. model. 
# => for lin.el. always set predict_D = False
# 

print(mat_tot)

As a check, in mat_tot (from sampling) (t1_tot =  5 mm) should be equal to t1 in mat_res (directly read) (t1_res =  30.0 mm).
      L     B    E_1  E_2   ms      F  F_N   s  t_1  t_2  nl  nu_1  nu_2  mat
0  1500  1500  70000  300  150  0.005    0  20    5  0.4   5  0.23   0.5   10


In [4]:
# DO NOT ADJUST THIS CODE BLOCK

samples = int(mat_tot.shape[0])
n_simple = len(desired_SN)

if simple:
    mat_res = [dict() for x in range(n_simple)]
    for i in range(int(n_simple)):
        mat = mat_tot.loc[i,:]
        mat_res[i] = main_solver(mat,conv_plt, NN_hybrid, path_collection)
        if i>0 and i%10 == 0:
            print('**********************************************************************')
            print('Data points upto row', i, 'are simulated')
            print('time required for first', i,'points:',time.time()-t0, 'secs') 

-------------------------------------------------------
1 Start Meshing with gmsh
-------------------------------------------------------
1.1 Assemble Model and mesh
1.2 Boundary Conditions and Loads
1.3 Read Material, Constitutive Model and Integration Options
1.4 Postprocess Mesh
   1.41 Node Coordinates
   1.42 Node Connectivity
   1.43 Element Center Information 1/2
   1.44 Element Numbering
   1.45 Global Coordinate Transformation
   1.46 Node Information
   1.47 Flip Element Arrangement Clockwise
   1.48 Element Center Information 2/2
   1.49 Element Connectivity
   1.410 Element Integration Point Information
   1.411 Geometrical Information per Element
   1.412 Material Information per Element
   1.413 Masks for Nodes in Areas
   1.414 Coplanar Nodes
-------------------------------------------------------
2 Solution
-------------------------------------------------------
2.1 Assembly of force vector and condensed DOFs
2.2 Solution for Linear Elasticity
Some values are outside th

c:\Users\vbalmer\Documents\GitHub\ShellSim\05_Deploying\NN_call.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(lit_model_path, map_lo

Gauss order for NN-run is:  1


c:\Users\vbalmer\Documents\GitHub\ShellSim\05_Deploying\Main_vb.py:575: RuntimeWarning: invalid value encountered in divide
  rele = np.divide(diffe,eh)


Some values are outside the lin.el. strain boundaries. Chosen model is still model III.
2.4 Solution complete
 - Maximum displacements:
   - ux_max = 0.0
   - uy_max = 0.0
   - uz_max = 4.762
 - Sum of applied forces:
   - Fx = 0.0
   - Fy = 0.0
   - Fz = -11250.0
 - Sum of reaction forces:
   - Rx = -38.7
   - Ry = -46.3
   - Rz = 6846.4
total time used:  19.171040296554565
time used in stress calculation 0
time used in strain calculation 0
time used in stiffness matrix calculation 1.5283691883087158
time used in B-matrix calculation 8.480615854263306
time used in stiffness matrix inversion 0
time used in calculation of sh 17.616660833358765
time used in calculation of eh 0.2383110523223877
-------------------------------------------------------
3 Postprocess Data
-------------------------------------------------------
Finished :)


In [5]:
if NN_hybrid['predict_sig'] and NN_hybrid['predict_D']: 
    fname = 'mat_res_NN.pkl'
elif NN_hybrid['predict_sig']:
    fname = 'mat_res_NN_sig.pkl'
elif NN_hybrid['predict_D']:
    fname = 'mat_res_NN_D.pkl'
else: 
    fname = 'mat_res_norm.pkl'

mat_res_pd_NN = pd.DataFrame.from_dict(mat_res)
mat_res_pd_NN.to_pickle(os.path.join('data_out',fname))
mat_res_pd_NN[['ms', 'L', 'B']].head()

,ms,L,B
0,150.0,1500.0,1500.0


In [6]:
# print(mat_res_pd_NN['uz'][0])
# print(mat_res_pd_NN['thx'][0])

In [7]:
# run the same simulation but without the NN (--> ground truth)
NN_hybrid_2 = {'predict_D': False,
             'predict_sig': False}                       # If true, solves the system with NN_hybrid solver; if False: "normal" solver
samples = int(mat_tot.shape[0])
n_simple = len(desired_SN)


if simple:
    mat_res = [dict() for x in range(n_simple)]
    for i in range(int(n_simple)):
        mat = mat_tot.loc[i,:]
        # with HiddenPrints():
        mat_res[i] = main_solver(mat,conv_plt, NN_hybrid_2, model_path)
        if i>0 and i%10 == 0:
            print('**********************************************************************')
            print('Data points upto row', i, 'are simulated')
            print('time required for first', i,'points:',time.time()-t0, 'secs') 

-------------------------------------------------------
1 Start Meshing with gmsh
-------------------------------------------------------
1.1 Assemble Model and mesh
1.2 Boundary Conditions and Loads
1.3 Read Material, Constitutive Model and Integration Options
1.4 Postprocess Mesh
   1.41 Node Coordinates
   1.42 Node Connectivity
   1.43 Element Center Information 1/2
   1.44 Element Numbering
   1.45 Global Coordinate Transformation
   1.46 Node Information
   1.47 Flip Element Arrangement Clockwise
   1.48 Element Center Information 2/2
   1.49 Element Connectivity
   1.410 Element Integration Point Information
   1.411 Geometrical Information per Element
   1.412 Material Information per Element
   1.413 Masks for Nodes in Areas
   1.414 Coplanar Nodes
-------------------------------------------------------
2 Solution
-------------------------------------------------------
2.1 Assembly of force vector and condensed DOFs
2.2 Solution for Linear Elasticity
Gauss order for NLFEA-run 

In [8]:
# Save relevant parameters
if NN_hybrid_2['predict_sig'] and NN_hybrid_2['predict_D']: 
    fname = 'mat_res_NN.pkl'
elif NN_hybrid_2['predict_sig']:
    fname = 'mat_res_NN_sig.pkl'
elif NN_hybrid_2['predict_D']:
    fname = 'mat_res_NN_D.pkl'
else: 
    fname = 'mat_res_norm.pkl'

mat_res_pd = pd.DataFrame.from_dict(mat_res)
mat_res_pd.to_pickle(os.path.join('data_out',fname))

mat_res_pd.head()
mat_res_pd[['ms', 'L', 'B']].head()

,ms,L,B
0,150.0,1500.0,1500.0


In [9]:
# if data should be saved to folder instead of being overwritten with the next simulation, use save_folder = True

from datetime import datetime
import shutil

save_folder = True

if save_folder:
    if NN_hybrid['predict_sig'] and NN_hybrid['predict_D']:
        relative_path = ['data_out\\mat_res_norm.pkl', 'data_out\\mat_res_NN.pkl']
    elif NN_hybrid['predict_sig']:
        relative_path = ['data_out\\mat_res_norm.pkl', 'data_out\\mat_res_NN_sig.pkl']
    elif NN_hybrid['predict_D']:
        relative_path = ['data_out\\mat_res_norm.pkl', 'data_out\\mat_res_NN_D.pkl']
    
    for i in range(len(relative_path)):
        source_folder = os.path.dirname(relative_path[i])
        file_name = os.path.basename(relative_path[i])

        current_time = datetime.now()
        new_folder = current_time.strftime("data_%Y%m%d_%H%M_case"+'xx')
        new_folder_path = os.path.join(source_folder, new_folder)

        os.makedirs(new_folder_path, exist_ok=True)

        destination_path = os.path.join(new_folder_path, file_name)
        shutil.copy(relative_path[i], destination_path)

        print('File', i, 'is copied to', destination_path)

File 0 is copied to data_out\data_20250207_1154_casexx\mat_res_norm.pkl
File 1 is copied to data_out\data_20250207_1154_casexx\mat_res_NN_sig.pkl


In [10]:
# print(mat_res_pd.keys())
# print(mat_res_pd['eps_g'][0].shape)
# print(mat_res_pd['eps_g'][0][:,0,0,0]) # these should correspond to eps_x
# print(mat_res_pd['eps_g'][0][:,0,0,7]) # these should correspond to gamma_xz
print(mat_res_pd['ms'])
print(mat_res_pd['t_1'])

0    150.0
Name: ms, dtype: float64
0    5.0
Name: t_1, dtype: float64


In [11]:
# Batchwise iteration not necessarily required for deployment


# iterate batchwise over all input sets (here: 3 batches which are created automatically based on size of input vector)
# else:
#     mat_res = [dict() for x in range(samples)]
#     for i in range(int(samples/3)):
#         mat = mat_tot.loc[i,:]
#         with HiddenPrints():
#             mat_res[int(i)] = main_solver(mat,conv_plt, NN_hybrid, model_path)
#         if i>0 and i%10 == 0:
#             print('**********************************************************************')
#             print('Data points upto row', i, 'are simulated')
#             print('time required for first', i,'points:',time.time()-t0, 'secs')
        
#     for i in np.linspace(
#         int(samples/3), 
#         2*int(samples/3), 
#         (2*int(samples/3)-int(samples/3))+1
#         ):
#         mat = mat_tot.loc[i,:]
#         with HiddenPrints():
#             mat_res[int(i)] = main_solver(mat,conv_plt, NN_hybrid, model_path)
#         if i%10 == 0:
#             print('**********************************************************************')
#             print('Data points upto row', i, 'are simulated')
#             print('time required for first', i, 'points:',time.time()-t0, 'secs')

#     for i in np.linspace(
#         2*int(samples/3), 
#         int(samples), 
#         (2*int(samples/3)-int(samples/3))+1
#         ):
#         mat = mat_tot.loc[int(i-1),:]
#         with HiddenPrints():
#             mat_res[int(i-1)] = main_solver(mat,conv_plt, NN_hybrid, model_path)
#         if (i-1)%10 == 0:
#             print('**********************************************************************')
#             print('Data points upto row', i-1, 'are simulated')
#             print('time required for first', i-1, 'points:',time.time()-t0, 'secs')